# Lab 4 – LoRA Fine-Tuning
**Part 4 of the 7 Lab Hands-On SLM Training Series**

In this lab, we move from preparing our dataset to teaching the Small Language Model (SLM) how to adapt to our domain. We do this using **LoRA (Low-Rank Adaptation)**, a parameter-efficient fine-tuning technique supported by the `unsloth` library.

By the end of this lab you will have:
- Attached LoRA adapters to a base model
- Run a short domain-adaptive training loop
- Verified that the model is adapting to your dataset

## Step 0. Install dependencies

In [ ]:
!pip install -q unsloth transformers datasets accelerate bitsandbytes


## Step 1. Load the prepared dataset from Google Drive

In [ ]:
from datasets import load_from_disk
from google.colab import drive
drive.mount('/content/drive')

# Load the tokenized dataset created in Lab 3 (saved to Google Drive)
DATA_DIR = "/content/drive/MyDrive/slm-labs/lab3_tokenized"
dataset = load_from_disk(DATA_DIR)
print(dataset)

## Step 2. Load the base model

In [ ]:
from unsloth import FastLanguageModel

model_name = "HuggingFaceH4/zephyr-7b-beta"  # replace with your base SLM if preferred

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model and tokenizer ready.")

## Step 3. Attach LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)
print("LoRA adapters attached.")

## Step 4. Fine-tune with LoRA

In [ ]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=50,  # keep small for demo
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_strategy="no",
    output_dir="./outputs",
)

trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset.get("test"),
)

trainer.train()

## Step 5. Save LoRA adapters

In [ ]:
model.save_pretrained("./lora_adapters")
tokenizer.save_pretrained("./lora_adapters")
print("LoRA adapters saved to ./lora_adapters")

### Wrap-up
In this lab you have:
- Loaded your domain dataset from Lab 3 (via Google Drive)
- Attached LoRA adapters to your base model
- Run a short fine-tuning loop
- Saved the resulting adapters

These adapters can now be reapplied to the base model any time you want, making fine-tuning lightweight and reusable.

Next up: **Lab 5 – Hyperparameter Tuning and Optimization**.